## Part IV:  Features and Labeling Functions

## Generating Features

In [1]:
# If necessary:
import os
os.remove('snorkel.db');
os.system('cp snorkel.db\ labels snorkel.db');

In [2]:
%load_ext autoreload
%autoreload 2

from snorkel import SnorkelSession
session = SnorkelSession()

In [3]:
from snorkel.models import candidate_subclass

Part_Temp = candidate_subclass('Part_Temp', ['part','temp'])

### Loading the `CandidateSet`

In [4]:
from snorkel.models import CandidateSet
train = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Training Candidates').one()
dev = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Development Candidates').one()

### Creating feature matrix

In [5]:
from snorkel.annotations import FeatureManager

feature_manager = FeatureManager()

In [6]:
%time F_train = feature_manager.create(session, train, 'Train Features')

[========================================] 100%

Loading sparse Feature matrix...
CPU times: user 45min 37s, sys: 1min 43s, total: 47min 21s
Wall time: 48min 8s


In [7]:
%time F_dev = feature_manager.update(session, dev, 'Train Features', expand_key_set=False)

[========================================] 100%

Loading sparse Feature matrix...
CPU times: user 5min 15s, sys: 24.8 s, total: 5min 40s
Wall time: 5min 40s


### Reloading feature matrix

In [8]:
%time F_train = feature_manager.load(session, train, 'Train Features')

CPU times: user 24.3 s, sys: 824 ms, total: 25.1 s
Wall time: 25.4 s


In [9]:
F_train

<11549x19955 sparse matrix of type '<type 'numpy.float64'>'
	with 1856014 stored elements in Compressed Sparse Row format>

In [10]:
F_train.get_candidate(0)

Part_Temp(Span("BC807K", parent=27872, chars=[94,99], words=[5,5]), ImplicitSpan("-55", parent=87287, words=[0,0], position=[0]))

In [11]:
F_train.get_key(0)

AnnotationKey (TDL_INV_LEMMA:PARENTS-OF-BETWEEN-MENTION-and-MENTION[None])

In [12]:
from snorkel.utils import get_keys_by_candidate

for f in get_keys_by_candidate(F_train, F_train.get_candidate(1))[:10]: print f

AnnotationKey (TAB_e1_W_RIGHT_1_[/])
AnnotationKey (TAB_e1_W_RIGHT_POS_1_[:])
AnnotationKey (TAB_e1_STARTS_WITH_CAPITAL)
AnnotationKey (TAB_e1_LENGTH_1)
AnnotationKey (TAB_e1_HTML_TAG_p)
AnnotationKey (TAB_e1_HTML_ATTR_[style])
AnnotationKey (TAB_e1_HTML_ANC_TAG_[html])
AnnotationKey (TAB_e1_HTML_ANC_TAG_[body])
AnnotationKey (TAB_e1_HTML_ANC_TAG_[p])
AnnotationKey (TAB_e2_WORD_SEQ_[-55])


### Saving feature matrices

In [13]:
# If necessary:
import os
os.system('cp snorkel.db snorkel.db\ featurized');

## Applying LFs

In [ ]:
# If necessary:
import os
os.remove('snorkel.db');
os.system('cp snorkel.db\ featurized snorkel.db');

from snorkel import SnorkelSession
session = SnorkelSession()

from snorkel.models import candidate_subclass
Part_Temp = candidate_subclass('Part_Temp', ['part','temp'])

from snorkel.models import CandidateSet
train = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Training Candidates').one()

In [14]:
from snorkel.annotations import LabelManager

label_manager = LabelManager()

### Defining Labeling Functions

In [15]:
from snorkel.lf_helpers import *

LFs = []

# def LF_training_gold(c):
#     corpus = session.query(Corpus).filter(Corpus.name == 'Hardware Training').one()
#     training_gold_dict = get_gold_dict(filename, 'stg_temp_min', docs=corpus.documents)
#     return 1 if ((c[0].parent.document.name).upper(), 
#                  (c[0].get_span()).upper(), 
#                  (''.join(c[1].get_span().split())).upper()) in training_gold_dict else 0
# LFs.append(LF_training_gold)

# def LF_reckless_abandon(c):
#     return 1

# POSITIVE

def LF_to_right(c):
    return 1 if 'to' in get_right_ngrams(c.temp, window=1) else 0
LFs.append(LF_to_right)

def LF_tilde_right(c):
    return 1 if '~' in get_right_ngrams(c.temp, window=1) else 0
LFs.append(LF_tilde_right)

# def LF_through_right(c):
#     return 1 if len(set(['through','thru']).intersection(
#         set(get_right_ngrams(c.temp, window=1)))) > 0 else 0
# LFs.append(LF_through_right)

def LF_150_right(c):
    return 1 if '150' in get_right_ngrams(c.temp, window=3) else 0
LFs.append(LF_150_right)

def LF_storage_row(c):
    return 1 if 'storage' in get_row_ngrams(c.temp) else 0
LFs.append(LF_storage_row)

# def LF_tj_row(c):
#     return 1 if 'tj' in get_row_ngrams(c.temp) else 0
# LFs.append(LF_tj_row)

def LF_c_row(c):
    return 1 if 'c' in get_row_ngrams(c.temp) else 0
LFs.append(LF_c_row)

def LF_same_table(c):
    return 1 if same_table(c) else 0
LFs.append(LF_same_table)

# BOTH

def LF_tstg_row(c):
    return 1 if 'tstg' in get_row_ngrams(c.temp) else -1
LFs.append(LF_tstg_row)

def LF_temperature_row(c):
    return 1 if 'temperature' in get_row_ngrams(c.temp) else -1
LFs.append(LF_temperature_row)

# NEGATIVE

def LF_temp_outside_table(c):
    return -1 if c.temp.parent.table is None else 0
LFs.append(LF_temp_outside_table)

def LF_complement_left(c):
    return -1 if 'complement' in get_left_ngrams(c.part) else 0
LFs.append(LF_complement_left)

def LF_voltage_row(c):
    return -1 if 'voltage' in get_row_ngrams(c.temp) else 0
LFs.append(LF_voltage_row)

### Testing LFs and LF_helpers

In [16]:
from snorkel.lf_helpers import get_right_ngrams, get_left_ngrams
from snorkel.lf_helpers import contains_token, contains_regex
from snorkel.lf_helpers import get_phrase_ngrams, get_cell_ngrams, get_neighbor_cell_ngrams
from snorkel.lf_helpers import get_row_ngrams, get_col_ngrams, get_aligned_ngrams
from snorkel.lf_helpers import same_document, same_table, same_cell, same_phrase
c = train[19]
print c.part
print c.temp
print c.temp.parent
print c.temp.parent.table
print same_document(c)
print same_table(c)
print contains_regex(c, rgx=r'849')
print LF_temp_outside_table(c)

Span("2N6426", parent=4159, chars=[0,5], words=[0,0])
ImplicitSpan("-55", parent=41460, words=[0,0], position=[0])
Phrase(Document MOTOS04796-1, 4171, None, 0, u'- 55  -  C TO 25  -  C')
Table(Document MOTOS04796-1, 6)
True
False
False
0


In [17]:
%time L_train = label_manager.create(session, train, 'LF Labels', f=LFs)
L_train

[========================================] 100%

Loading sparse Label matrix...
CPU times: user 3min 22s, sys: 16 s, total: 3min 38s
Wall time: 3min 33s


<11549x13 sparse matrix of type '<type 'numpy.float64'>'
	with 36706 stored elements in Compressed Sparse Row format>

In [18]:
train_gold = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Training Candidates -- Gold').one()

In [21]:
# print type(L_train)
# print L_train.lf_stats()
%time L_train.lf_accuracy(train_gold)

,accuracy,conflicts,coverage,j,overlaps
LF_to_right,0.668076,0.079834,0.122868,0,0.122868
LF_tilde_right,0.228669,0.024851,0.025370,1,0.025370
LF_through_right,0.000000,0.000000,0.000000,2,0.000000
LF_150_right,0.444737,0.065547,0.065807,3,0.065807
LF_storage_row,0.506390,0.188155,0.250671,4,0.250671
LF_tj_row,0.000000,0.000000,0.000000,5,0.000000
LF_c_row,0.361574,0.347043,0.431293,6,0.431293
LF_same_table,0.214600,0.090311,0.098450,7,0.098450
LF_tstg_row,0.775305,0.444887,1.000000,8,1.000000
LF_temperature_row,0.801108,0.444887,1.000000,9,1.000000


In [20]:
# If necessary
import os
os.system('cp snorkel.db snorkel.db\ features');

Next, in Part 5, we will test our model on the development `CandidateSet`.